First import necessary libraries and read the input data. We're going to do some explorations on the data to understand it better.


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#remove the date columns
# we can't really use them for anything as pertains to predicting the outcome
# at least that's what I think
x = pd.read_csv('./traininginputs.csv')
x.drop(columns=['PROC_TRACEINFO'],inplace=True)
y = pd.read_csv('./trainingoutput.csv')
y.drop(columns=['PROC_TRACEINFO'],inplace=True)

# we need to convert the Binar OP130_Resultat_Global_v to a boolean
y['Binar OP130_Resultat_Global_v'] = y['Binar OP130_Resultat_Global_v'].astype(
    'bool')

# we're just creating this to explore the data x & y is what will be actualy used
training = pd.concat([x, y['Binar OP130_Resultat_Global_v']], axis=1)

# training.describe(include='all')

Using the df.corr() function we can see how much the columns are related to the target

In [ ]:

training.corr(numeric_only=True)['Binar OP130_Resultat_Global_v'].sort_values(ascending=False)


No column has a signficantly high correlation with the target. So no liner models for this one.

Since there is little correlation, we have to use a non-linear model to predict the target. We will use a Random Forest Classifier, this can handle missing values, which we have and is less prone to overfitting like a single decision tree.

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
from numpy import ravel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import auc, roc_curve, accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42, shuffle=True)
#tweak these variables to get a better model. Good luck!
#you can also look up GridSearchCV to automate this process, ask ChatGPT
rf = RandomForestClassifier(class_weight='balanced', random_state=42, n_estimators=12,
                            max_depth=7, min_samples_leaf=0.05,max_features=0.65)
rf.fit(X_train, ravel(y_train))
y_pred = rf.predict(X_test)

# Evaluate Model
# Get probability scores for the positive class
y_probs = rf.predict_proba(X_test)[:, 1]

fpr, tpr, thresholds = roc_curve(y_test, y_probs)
rf_auc = auc(fpr, tpr)  # Compute AUC Score

print(f'Random Forest Classifier Accuracy: {rf_auc}') #should be 0.685, try for 0.7
print(f'Random Forest Classifier Accuracy: {accuracy_score(y_test, y_pred)}') #not so important here, but its 0.742

Random Forest Classifier Accuracy: 0.6849951079030442
Random Forest Classifier Accuracy: 0.7418009039286129


We should try to make something like the sample submission file, using our model. But you can do that, hmm?